In [1]:
import numpy as np

# ReLU activation function
def ReLU(x):
    return np.maximum(0, x)

# Derivative of ReLU
def ReLU_derivative(x):
    return np.where(x > 0, 1, 0)

# Softmax activation function
def Softmaxfxn(x):
    exp_x = np.exp(x - np.max(x, axis=0, keepdims=True))  # Stabilize for numerical safety
    return exp_x / np.sum(exp_x, axis=0, keepdims=True)

# Cross-entropy loss function
def cross_entropy_loss(y_true, y_pred):
    # Add a small value (epsilon) to prevent log(0)
    epsilon = 1e-12
    y_pred = np.clip(y_pred, epsilon, 1. - epsilon)
    return -np.mean(np.sum(y_true * np.log(y_pred), axis=0))


# Gradient of the cross-entropy loss
def loss_gradient(y_true, y_pred):
    return y_pred - y_true


In [2]:
import numpy as np

class Layer:
    def __init__(self, input_size, num_neurons, activation_fxn):
        # Xavier Initialization
        limit = np.sqrt(6 / (input_size + num_neurons))
        self.weights = np.random.uniform(-limit, limit, (num_neurons, input_size))
        self.bias = np.zeros((num_neurons, 1))
        self.activation_fxn = activation_fxn


    def forward_propagation(self, inputs):
        self.a_prev = inputs  # Store inputs for backpropagation
        self.z = np.dot(self.weights, inputs) + self.bias  # Linear transformation
        self.a = self.activation_fxn(self.z)  # Apply activation function
        return self.a

    def gradient_calc(self, dl_dy_pred, weights_next, z, a_prev):
        """
    dl_dy_pred: Gradient of the loss with respect to the current layer's output
    weights_next: Weights of the next layer (used for propagating gradients)
    z: Pre-activation outputs of the current layer
    a_prev: Activations from the previous layer (input to the current layer)
     """
    # Compute gradient of loss with respect to the current layer's pre-activation output
        self.dl_dz = dl_dy_pred * ReLU_derivative(z)  # Element-wise multiplication

    # Compute gradients for the weights and biases
        dl_dw = np.dot(self.dl_dz, a_prev.T)  # Gradient w.r.t. weights
        dl_db = np.sum(self.dl_dz, axis=1, keepdims=True)  # Gradient w.r.t. biases

    # Compute the gradient to pass to the previous layer
        dl_da_prev = np.dot(self.weights.T, self.dl_dz)  # Gradient to pass to the previous layer

        return dl_da_prev, dl_dw, dl_db


In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report

class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers

    def forward_pass(self, inputs):
        for layer in self.layers:
            inputs = layer.forward_propagation(inputs)
        return inputs

    def backward_pass(self, expected, actual, learning_rate):
        # Compute the gradient of the loss with respect to the output
        loss_grad = loss_gradient(expected, actual)

        # Iterate through layers in reverse order (backpropagation)
        for layer in reversed(self.layers):
            # Call gradient_calc with the current layer's weights
            loss_grad, dl_dw, dl_db = layer.gradient_calc(loss_grad, None, layer.z, layer.a_prev)

            # Update the weights and biases using gradient descent
            layer.weights -= dl_dw * learning_rate
            layer.bias -= dl_db * learning_rate

# Define network structure
input_layer = Layer(784, 128, ReLU)  # Input layer
hidden_layer_1 = Layer(128, 64, ReLU)  # Hidden layer
output_layer = Layer(64, 10, Softmaxfxn)  # Output layer

nn = NeuralNetwork([input_layer, hidden_layer_1, output_layer])

# Training configuration
epochs = 50
learning_rate = 0.01
batch_size = 6

# Load and preprocess the dataset
df = pd.read_csv("/content/train.csv")

# Split dataset into training and testing subsets
df_randomized = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Limit to the first 700 rows for testing and the next 1000-1700 rows for training
test_df = df_randomized.head(700)
train_df = df_randomized.iloc[700:1700]
# Extract training data
train_inputs = train_df.iloc[:, 1:].values  # Input data
train_labels = train_df.iloc[:, 0].values  # Output labels

# Extract testing data
test_inputs = test_df.iloc[:, 1:].values  # Input data
test_labels = test_df.iloc[:, 0].values  # Output labels

# Normalize input data
train_inputs = train_inputs / 255.0  # Normalize pixel values to [0, 1]
test_inputs = test_inputs / 255.0  # Normalize pixel values to [0, 1]

# One-hot encode the training labels
num_classes = 10
train_expected_outputs = np.zeros((len(train_labels), num_classes))
for idx, label in enumerate(train_labels):
    train_expected_outputs[idx, label] = 1

# Training loop
for epoch in range(epochs):
    for i in range(0, len(train_inputs), batch_size):
        # Load batch data
        inputs = train_inputs[i:i+batch_size].T
        expected_output = train_expected_outputs[i:i+batch_size].T

        # Forward pass
        predicted_output = nn.forward_pass(inputs)

        # Backward pass
        nn.backward_pass(expected_output, predicted_output, learning_rate)

    # Calculate and print loss after each epoch
    total_loss = cross_entropy_loss(train_expected_outputs.T, nn.forward_pass(train_inputs.T))
    print(f"Epoch {epoch + 1}, Loss: {total_loss}")

# Testing phase
test_predictions = nn.forward_pass(test_inputs.T)  # Forward pass
predicted_labels = np.argmax(test_predictions, axis=0)  # Get the predicted labels

# Evaluate the model
accuracy = accuracy_score(test_labels, predicted_labels)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Detailed classification report
print("Classification Report:")
print(classification_report(test_labels, predicted_labels))


Epoch 1, Loss: 0.45036300768693754
Epoch 2, Loss: 0.24241360077388505
Epoch 3, Loss: 0.1540164922160518
Epoch 4, Loss: 0.1104867947188917
Epoch 5, Loss: 0.09295409774616363
Epoch 6, Loss: 0.06141307076347959
Epoch 7, Loss: 0.04364381145980653
Epoch 8, Loss: 0.02960921126815362
Epoch 9, Loss: 0.023274046486618162
Epoch 10, Loss: 0.01743067131578412
Epoch 11, Loss: 0.014556065758653586
Epoch 12, Loss: 0.012384396485430204
Epoch 13, Loss: 0.010801084755738877
Epoch 14, Loss: 0.009555266144449908
Epoch 15, Loss: 0.006542609036788166
Epoch 16, Loss: 0.004853919528918299
Epoch 17, Loss: 0.004035728814946066
Epoch 18, Loss: 0.0035015756565726914
Epoch 19, Loss: 0.0030810671301502064
Epoch 20, Loss: 0.002755353524674634
Epoch 21, Loss: 0.002509073275066234
Epoch 22, Loss: 0.0022867321389078705
Epoch 23, Loss: 0.002115651530329816
Epoch 24, Loss: 0.001961589524692625
Epoch 25, Loss: 0.0018244347185871672
Epoch 26, Loss: 0.001713407927200236
Epoch 27, Loss: 0.0016056384054670535
Epoch 28, Loss: 